## Function to create a list with all the file paths within a path

In [574]:
def getFilePaths(path):
    # Function to return a list of files in a path:
    # Initialize a list for all the file names
    files = list()
    # Look for folders in the list
    for file in os.listdir(path):
        # Concatenate a path for each file
        pathfile = os.path.join(path, file)
        # Validate if the pathfile is a folder
        if os.path.isdir(pathfile):
            files = files + getFilePaths(pathfile)
        else:
            files.append(pathfile)
    return files      

## Create a dataframe with the list of paths

In [575]:
# Import dependencies
import pandas as pd, time, hashlib

In [576]:
# Path with the files to look for duplicates
path='../Photos2'
# Initialize and empty dataframe with the paths of the files in the given path
files_df = pd.DataFrame(getFilePaths(path),columns=['path'])
# Initialize Columns for the dataframe 
files_df['hash'] = 0
files_df['filename'] = ''
files_df['count'] = 1
files_df['created'] = 0
files_df['modified'] = 0

# Loop to fill the dataframe columns with values
for row in range(len(files_df)):
    # Load the file
    image_file = open(files_df['path'][row],'rb').read()
    # Encode the file to a hexagsimal number
    files_df['hash'][row] = hashlib.md5(image_file).hexdigest()
    # Get the filename of the file
    files_df['filename'][row] = os.path.basename(files_df['path'][row])
    # Get the created date 
    files_df['created'][row] = pd.to_datetime(time.ctime(os.path.getctime(files_df['path'][row])))
    # Get the modified date 
    files_df['modified'][row] = pd.to_datetime(time.ctime(os.path.getmtime(files_df['path'][row])))
# Print first rows of the dataframe    
files_df.head()

,path,hash,filename,count,created,modified
0,../Photos2/.DS_Store,442550fe7605a4815b5cfcd2826a0b31,.DS_Store,1,2020-04-09 19:00:48,2020-04-09 19:00:48
1,../Photos2/New Folder With Items 2/7612e4ee-b4...,135a0a7b2f5eece7ce53c6fbb26e9cd9,7612e4ee-b49a-46d2-94d9-fbc234c71b2e.jpg,1,2020-04-09 19:00:43,2020-04-07 14:38:28
2,../Photos2/New Folder With Items 2/0009845a-53...,933827ccd64bd5345025069ddc79451f,0009845a-53ab-4b7b-838e-3e76fa6a638f copy 2.jpg,1,2020-04-09 19:00:43,2020-04-07 15:13:26
3,../Photos2/New Folder With Items/0009845a-53ab...,933827ccd64bd5345025069ddc79451f,0009845a-53ab-4b7b-838e-3e76fa6a638f.jpg,1,2020-04-09 19:00:38,2020-04-07 15:13:26
4,../Photos2/New Folder With Items/0009845a-53ab...,933827ccd64bd5345025069ddc79451f,0009845a-53ab-4b7b-838e-3e76fa6a638f copy 3.jpg,1,2020-04-09 19:00:38,2020-04-07 15:13:26


In [577]:
import numpy as np, datetime
reapeated_files_df = files_df.groupby('hash').sum()
reapeated_files_df = reapeated_files_df[reapeated_files_df['count'] > 1]
pivot_df = pd.pivot_table(reapeated_files_df, values='count', index=['hash'], aggfunc=np.sum, margins=True)
now_string = datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S")
pivot_df.to_csv('repeated_' + now_string + '.csv' )
pivot_df

,count
hash,
933827ccd64bd5345025069ddc79451f,4
All,4


In [578]:
repeated_path_df = files_df[files_df['hash'].isin(pivot_df.index.to_list())]
repeated_path_df = repeated_path_df.groupby(['hash','path','created','modified']).sum().sort_values(by=['hash', 'created', 'modified'], ascending=True)
repeated_path_df.to_csv('repeated_list' + now_string + '.csv' )
repeated_path_df



count
hash                             path                                               created             modified                  
933827ccd64bd5345025069ddc79451f ../Photos2/New Folder With Items/0009845a-53ab-... 2020-04-09 19:00:38 2020-04-07 15:13:26      1
                                 ../Photos2/New Folder With Items/0009845a-53ab-... 2020-04-09 19:00:38 2020-04-07 15:13:26      1
                                 ../Photos2/New Folder With Items/0009845a-53ab-... 2020-04-09 19:00:38 2020-04-07 15:13:26      1
                                 ../Photos2/New Folder With Items 2/0009845a-53a... 2020-04-09 19:00:43 2020-04-07 15:13:26      1

In [579]:
range(1,len(repeated_path_df.index.get_level_values(1).tolist()))

range(1, 4)

In [580]:

repeated_path_df['action'] = ''
repeated_path_df['action'][0] = 'leave' 


for i in range(1,len(repeated_path_df.index.get_level_values(1).tolist())):
    if repeated_path_df.index.get_level_values(0)[i] == repeated_path_df.index.get_level_values(0)[i-1]:
        repeated_path_df['action'][i] = 'delete' 
    else:
        repeated_path_df['action'][i] = 'delete' 

repeated_path_df        

count  \
hash                             path                                               created             modified                     
933827ccd64bd5345025069ddc79451f ../Photos2/New Folder With Items/0009845a-53ab-... 2020-04-09 19:00:38 2020-04-07 15:13:26      1   
                                 ../Photos2/New Folder With Items/0009845a-53ab-... 2020-04-09 19:00:38 2020-04-07 15:13:26      1   
                                 ../Photos2/New Folder With Items/0009845a-53ab-... 2020-04-09 19:00:38 2020-04-07 15:13:26      1   
                                 ../Photos2/New Folder With Items 2/0009845a-53a... 2020-04-09 19:00:43 2020-04-07 15:13:26      1   

                                                                                                                             action  
hash                             path                                               created             modified                     
933827ccd64bd5345025069ddc79451f ../Photos2/New Folder With Items/0009845a-53ab-... 2020-04-09 19:00:38 2020-04-07 15:13:26   leave  
                                 ../Photos2/New Folder With Items/0009845a-53ab-... 2020-04-09 19:00:38 2020-04-07 15:13:26  delete  
                                 ../Photos2/New Folder With Items/0009845a-53ab-... 2020-04-09 19:00:38 2020-04-07 15:13:26  delete  
                                 ../Photos2/New Folder With Items 2/0009845a-53a... 2020-04-09 19:00:43 2020-04-07 15:13:26  delete

In [581]:
len(repeated_path_df.index.get_level_values(1).tolist())




4

In [582]:
# Delete list
for path in repeated_path_df.index.get_level_values(0).unique().to_list():
    print(path)
    

933827ccd64bd5345025069ddc79451f
